# GMNS/Network Wrangler 2.0 Workshop

***Placeholder text from workshop overview:*** *Building and maintaining transportation networks for travel models is often time-consuming and error-prone. This workshop introduces the General Modeling Network Specification (GMNS), a standardized, human- and machine-readable format developed by the Zephyr Foundation with support from FHWA. GMNS enables easier sharing of routable road network files across platforms and agencies.  Participants will also explore Network Wrangler 2.0, an open-source Python tool that simplifies network editing through “Project Cards” - standardized, human- and machine-readable text files that describe proposed changes to roadway, transit, pedestrian, or bicycle infrastructure. The tool automates network updates, supports scenario management, and facilitates collaboration across agencies and software platforms.  This hands-on workshop will guide participants through building GMNS- and Network Wrangler-ready travel model networks from scratch using open-source data. Attendees will learn how to create network scenarios with Network Wrangler and perform network analysis with GMNS standard networks, including transit accessibility evaluation. Live demonstrations will showcase how these tools streamline network development and scenario management for regional transportation planning.*  


## Workflow Overview:

*Insert updata workflow*

In [ ]:
import IPython
from IPython.display import display
IPython.core.display.display_html('<script>IPython.notebook.trusted = true;</script>', raw=True)

import argparse
import datetime
import pathlib
import pickle
import pprint
import requests
import statistics
import sys
from typing import Any, Optional, Tuple, Union

import networkx
import osmnx
import numpy as np
import pandas as pd
import geopandas as gpd
import pygris
import shapely.geometry

import folium
from folium import plugins
import seaborn as sns

import tableau_utils
import network_wrangler
from network_wrangler import WranglerLogger
from network_wrangler.params import LAT_LON_CRS
from network_wrangler.roadway.network import RoadwayNetwork
from network_wrangler.roadway.io import load_roadway_from_dataframes, write_roadway
from network_wrangler.roadway.clip import clip_roadway, clip_roadway_to_dfs
from network_wrangler.models.gtfs.gtfs import GtfsModel
from network_wrangler.transit.feed.feed import Feed
from network_wrangler.transit.network import TransitNetwork
from network_wrangler.transit.io import load_feed_from_path, write_transit, load_transit
from network_wrangler.models.gtfs.types import RouteType
from network_wrangler.utils.transit import \
  drop_transit_agency, filter_transit_by_boundary, create_feed_from_gtfs_model
from network_wrangler.roadway.centroids import FitForCentroidConnection, add_centroid_nodes, add_centroid_connectors

from visualization import *

import sys
sys.path.append('../create_baseyear_network')
from create_mtc_network_from_OSM import(
    get_travel_model_zones,
    stepa_standardize_attributes,
    step1_download_osm_network,
    step2_simplify_network_topology,
    step3_assign_county_node_link_numbering,
    step4_add_centroids_and_connectors,
    step5_prepare_gtfs_transit_data,
    step6_create_transit_network,
)

In [ ]:
OUTPUT_DIR = pathlib.Path("output_OSM").resolve()
OUTPUT_DIR.mkdir(exist_ok=True)
INPUT_2023GTFS = pathlib.Path("M:/Data/Transit/511/2023-09")

import getpass
if getpass.getuser()=="lmz":
    INPUT_2023GTFS = (OUTPUT_DIR / ".." / ".." / ".." / "511gtfs_2023-09").resolve()
    print(INPUT_2023GTFS)

from network_wrangler import WranglerLogger
import pathlib
info_log_file = OUTPUT_DIR / "create_SF_network_info.log"
debug_log_file = OUTPUT_DIR / "create_SF_network_debug.log"
network_wrangler.setup_logging(
    info_log_file,
    debug_log_file,
    std_out_level="info",
    file_mode="w"
  )

# We have custom loggers and we want to prevent their messages from being
# processed by the root logger's handlers (if any remain)
WranglerLogger.propagate = False

# this one will just go to the debug file
WranglerLogger.debug("Debug test")
# this will go to the console (stdout) and the info & debug files
WranglerLogger.info("Info test")

In [ ]:
# fetch travel model zones; this will save into OUTPUT_DIR/mtc_zones
travel_model_zones = get_travel_model_zones(OUTPUT_DIR)
print(travel_model_zones["TAZ"])

## Step 1: Download OSM network data
Gets raw road network from OpenStreetMap

In [ ]:
# Download the OSM network data for San Francisco county
osm_g = step1_download_osm_network(county="San Francisco", output_dir=OUTPUT_DIR) 

In [ ]:
# Visualize the network
fig, ax = create_osmnx_plot(osm_g)

## Step 1A: Standardize attributes (and write)  
Note: we don't keep the results of this, since we'll use version from the simplified graph  
*VIZ: visualize complicated block - van ness with bus lanes and footway a couple blocks north of market*

In [ ]:
links_unsimplified_gdf, nodes_unsimplified_gdf = stepa_standardize_attributes(
    osm_g, 
    county="San Francisco", 
    prefix="1a_original_", 
    output_dir=OUTPUT_DIR,
    output_formats=["geojson"]
)

## Step 2: Simplify network topology  
Consolidates intersections while preserving connectivity

In [ ]:
simplified_g = step2_simplify_network_topology(osm_g, county="San Francisco", output_dir=OUTPUT_DIR)

In [ ]:
compare_original_and_simplified_networks(osm_g, simplified_g)
plot_node_degree_changes(osm_g, simplified_g)

## Step 2A Standardize attributes and write
*VIZ: same as 1a for contrast*

In [ ]:
links_gdf, nodes_gdf = stepa_standardize_attributes(
    simplified_g, county="San Francisco", 
    prefix="2a_simplified", 
    output_dir=OUTPUT_DIR,
    output_formats=["geojson"]
)

In [ ]:
links_unsimplified_clip_gdf, links_clip_gdf = clip_original_and_simplified_links(links_unsimplified_gdf, links_gdf, travel_model_zones["TAZ"])
print(f"links_unsimplified_clip_gdf has length {len(links_unsimplified_clip_gdf)}")
print(f"links_clip_gdf has length {len(links_clip_gdf)}")

In [ ]:
m = map_original_and_simplified_links(links_unsimplified_clip_gdf, links_clip_gdf)
m

In [ ]:
links_unsimplified_gdf["highway"].unique()

## Step 3: Assign county-specific numbering and create RoadwayNetwork object  
This also drops columns we're done with and writes the roadway network

In [ ]:
roadway_network = step3_assign_county_node_link_numbering(links_gdf, nodes_gdf, county="San Francisco", output_dir=OUTPUT_DIR, output_formats=["geojson"])

## Step 4: Add centroids and centroid connectors  

In [ ]:
step4_add_centroids_and_connectors(roadway_network, county="San Francisco", output_dir=OUTPUT_DIR, output_formats=["geojson"])

In [ ]:
m = create_roadway_network_map(roadway_network.links_df)
m

## STEP 5: Prepare GTFS transit data: Read and filter to service date, relevant operators. Creates GtfsModel object  
This also writes the GtfsModel as GTFS

In [ ]:
gtfs_model = step5_prepare_gtfs_transit_data(county="San Francisco", input_gtfs=INPUT_2023GTFS, output_dir=OUTPUT_DIR)

# STEP 6: Create TransitNetwork by integrating GtfsModel with RoadwayNetwork to create a Wrangler-flavored Feed object  
This writes the RoadwayNetwork and TransitNetwork

In [ ]:
transit_network = step6_create_transit_network(gtfs_model, roadway_network, county="San Francisco", output_dir=OUTPUT_DIR, output_formats=["parquet"])

In [ ]:
# visualize, now with rail links
m = create_roadway_network_map(roadway_network.links_df)
m

In [ ]:

# print(f"transit_network.feed.shapes.columns: {transit_network.feed.shapes.columns}")
shape_links_gdf = transit_network.feed.shapes.sort_values(by=["shape_id","shape_pt_sequence"]).reset_index(drop=True)
shape_links_gdf["next_shape_model_node_id"] = shape_links_gdf.groupby("shape_id")["shape_model_node_id"].shift(-1)
shape_links_gdf["next_shape_pt_sequence"] = shape_links_gdf.groupby("shape_id")["shape_pt_sequence"].shift(-1)
# Filter to only rows that have a next node (excludes last point of each shape)
shape_links_gdf = shape_links_gdf.loc[ shape_links_gdf["next_shape_model_node_id"].notna() ]
shape_links_gdf["next_shape_model_node_id"] = shape_links_gdf["next_shape_model_node_id"].astype(int)
shape_links_gdf["next_shape_pt_sequence"] = shape_links_gdf["next_shape_pt_sequence"].astype(int)
shape_links_gdf.rename(columns={"shape_model_node_id":"A", "next_shape_model_node_id":"B"}, inplace=True)
# display(shape_links_gdf.loc[ shape_links_gdf.shape_id=="SF:1400:20230930", ["route_type","route_id","shape_pt_sequence","next_shape_pt_sequence","A","B"]])

# drop these columns
shape_links_gdf.drop(columns=["shape_id","shape_pt_lat","shape_pt_lon","geometry","stop_id","stop_name"], inplace=True)
# print(f"shape_links_gdf.columns: {shape_links_gdf.columns}")

# get road network shapes
roadnet_shapes_gdf = roadway_network.links_df
# print(f"roadnet_shapes_gdf.columns:{roadnet_shapes_gdf.columns}")

# join to roadway_links.shape_df
shape_roadnet_links_gdf = gpd.GeoDataFrame(
    pd.merge(
        left=shape_links_gdf,
        right=roadnet_shapes_gdf,
        on=["A","B"],
        how="left",
        indicator=True
    ), crs=roadnet_shapes_gdf.crs
)
print(f"Route ids: {shape_roadnet_links_gdf["route_id"].unique()}")

create_roadway_transit_map(roadway_network.links_df, shape_roadnet_links_gdf, route_ids=["SF:1","SF:9","SF:28R"])